<a href="https://colab.research.google.com/github/Akshay00713/ML/blob/main/crime_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Git if not already installed
!apt-get install git

# Configure Git
!git config --global user.name "Akshay00713"
!git config --global user.email "akshaymairal13@gmail.com"


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
# Clone your repository
!git clone https://github.com/Akshay00713/ML.git

# Navigate to the repository folder
%cd ML


Cloning into 'ML'...
/content/ML


In [ ]:
# Move your Colab file into the repository folder
# !cp /content/Crime prediction.ipynb

# # Navigate to the repository folder
# %cd /content/<repository-name>

# # Add the notebook to the repository
!git add .

# Commit changes
!git commit -m "Added Colab notebook"

# Push changes to GitHub
!git push origin main

On branch main

Initial commit

nothing to commit (create/copy files and use "git add" to track)
error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/Akshay00713/ML.git'


In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib
import tkinter as tk
from tkinter import ttk

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
# Bounus
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
# Metrics
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,classification_report, confusion_matrix, roc_auc_score

ModuleNotFoundError: No module named 'catboost'

In [ ]:
df = pd.read_csv('Crime Prediction in Chicago_Dataset.csv')

In [ ]:
df.head(10).T

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df['Location Description'].value_counts().head(10)

In [ ]:
df['Location Description'] = df['Location Description'].fillna(df['Location Description'].mode()[0])

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df_null_ward = df[df.Ward.isnull()]
df_null_ward.T

In [ ]:
df_null_ward['District']

In [ ]:
df2=df[df['District']==24]
df2['Ward'].describe()

In [ ]:
df2 = df[df['District']==16]
df2['Ward'].describe()

In [ ]:
df.loc[df['District'] == 24, 'Ward'] = 49
df.loc[df['District'] == 16, 'Ward'] = 41

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(subset=['X Coordinate'], inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df.Date

In [ ]:
df.Block.value_counts().head(10)

In [ ]:
df.info()

In [ ]:
df['IUCR'].value_counts().head(10)

In [ ]:
df.info()

In [ ]:
count_prim = df['Primary Type'].value_counts().head(15)
count_prim

In [ ]:
plt.style.use("fivethirtyeight")
plt.figure(figsize=(8,5))
sns.barplot(y=count_prim.index,x=count_prim.values,palette="viridis")
plt.title("TOP 15 Primary Types")
plt.ylabel("Types")
plt.xlabel("count")
plt.show()

In [ ]:
count_Des = df['Description'].value_counts().head(10)
count_Des

In [ ]:
plt.style.use("fivethirtyeight")
plt.figure(figsize=(6,4))
sns.barplot(y=count_Des.index,x=count_Des.values,palette="viridis")
plt.title("TOP 5 Descriprtion")
plt.ylabel("Description")
plt.xlabel("count")
plt.xticks(rotation=45)
plt.show()

In [ ]:
count = df['Arrest'].value_counts()
percentage = df['Arrest'].value_counts(normalize=True)*100
freqTable = pd.DataFrame({'Frequency':count,'Percentage':percentage})
freqTable

In [ ]:
plt.figure(figsize=(8, 4))
sns.heatmap(freqTable.T, annot=True, fmt=".1f", cmap="YlGnBu", cbar=False)
plt.title("Arrest")
plt.xlabel('')
plt.show()

In [ ]:
count_dom = df['Domestic'].value_counts()
count_dom

In [ ]:
plt.style.use("ggplot")
plt.figure(figsize=(6, 6))
plt.pie(count_dom, labels=count_dom.index, autopct='%1.1f%%', startangle=90,textprops={"fontweight":"black"},explode=[0, 0.1])
plt.title("Domestic")
plt.legend()
plt.show()

In [ ]:
df['Beat'].value_counts()

In [ ]:
df['District'].value_counts()

In [ ]:
df['Ward'].value_counts().head(20)

In [ ]:
df['Community Area'].value_counts().head(10)

In [ ]:
df['FBI Code'].value_counts()

In [ ]:
df = df.drop(columns=['Year', 'Location'])

In [ ]:
df.groupby(['IUCR', 'Primary Type', 'FBI Code']).size().head(20)

In [ ]:
df.groupby(['Primary Type'])['Description'].value_counts().head(50)

In [ ]:
df.groupby(['Description'])['Arrest'].value_counts(normalize=True).head(10)*100

In [ ]:
df.groupby(['Primary Type'])['Arrest'].value_counts(normalize=True).tail(10)*100

In [ ]:
df.groupby(['Location Description'])['Arrest'].value_counts(normalize=True).head(20)*100

In [ ]:
def splitBlock(x):
    x = x.split(" ")
    x = x[1:]
    return " ".join(x[0:])
df.Block = df.Block.apply(splitBlock)

In [ ]:
columns = df[['Description','Primary Type','Location Description','Block','Date','Time']]
encoders={}
for col in columns:
    encoder = LabelEncoder()
    df[col] = encoder.fit_transform(df[col])
    encoders[col] = encoder

In [ ]:
df.info()

In [ ]:
correlation_matrix = df.corr(numeric_only=True)
correlation_arrest = correlation_matrix['Arrest'].sort_values(ascending=False)
plt.figure(figsize=(8, 5))
sns.heatmap(correlation_arrest.to_frame(), annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation with Arrest')
plt.show()

In [ ]:
X = df.drop(columns=['ID','Arrest','Case Number','X Coordinate','Y Coordinate','Latitude','Longitude','IUCR','FBI Code','Community Area','Updated On'])
y = df['Arrest']

In [ ]:
X

In [ ]:
smote = SMOTE(sampling_strategy=0.75,random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

In [ ]:
y_resampled.value_counts()

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_resampled,y_resampled,test_size=0.3,shuffle=True,random_state=57)

In [ ]:
models = {
    'LogisticRegression': LogisticRegression(max_iter=500),
    'Decision Tree Classifier': DecisionTreeClassifier(splitter='best',random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
    'Random Forest': RandomForestClassifier(n_estimators=200, random_state=57),
    'XGBoost': XGBClassifier(n_estimators=100,learning_rate=0.01),
    'CatBoost': CatBoostClassifier(learning_rate=0.08,depth=10,l2_leaf_reg=1.7,iterations=500, verbose=0),
    'LightGBM': LGBMClassifier(n_estimators=100, learning_rate=0.01,verbose=0)
}

results = []

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]

    print(model_name)
    print("-"*10)

    confusion = confusion_matrix(y_test, y_pred)
    accuracy = round(accuracy_score(y_test, y_pred) * 100, 2)
    precision = round(precision_score(y_test, y_pred) * 100, 2)
    recall = round(recall_score(y_test, y_pred) * 100, 2)
    f1 = round(f1_score(y_test, y_pred) * 100, 2)
    roc = round(roc_auc_score(y_test, y_pred_prob) * 100, 2)

    plt.figure(figsize=(6, 4))
    sns.heatmap(confusion, annot=True, fmt='d', cmap='Blues', linewidths=.5, cbar=False)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

    print(classification_report(y_test, y_pred))
    results.append({
        'Model': model_name,
        'Accuracy Score': f"{accuracy}%",
        'Precision': f"{precision}%",
        'Recall': f"{recall}%",
        'F1 Score': f"{f1}%",
        'ROC-AUC': f"{roc}%"
    })

results_df = pd.DataFrame(results)
display(results_df)

In [ ]:
model = RandomForestClassifier(n_estimators=200, random_state=57)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_pred_prob = model.predict_proba(X_test)[:, 1]


acc = accuracy_score(y_test,y_pred)
confusion = confusion_matrix(y_test,y_pred)
classification_rep = classification_report(y_test,y_pred)
roc = roc_auc_score(y_test,y_pred_prob)


print(f'{round(acc*100)}%')
print(confusion)
print(classification_rep)
print(f'{round(roc*100)}%')


plt.figure(figsize=(6, 4))
sns.heatmap(confusion, annot=True, fmt='d', cmap='Blues', linewidths=.5, cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()